# Set Partitioning Problem
The set partitioning problem is an optimization problem which selects sets $S_i$ from a collection $S$ such that each member $x\in X=\bigcup_i S_i$ is included in exactly one $S_i$ of the selected sets (see [Operations Research Journal Vol. 17, No. 5](https://doi.org/10.1287/opre.17.5.848)). It has applications in logistics, design, manufacturing and scheduling, among others.





## Formulation
The set partitioning problem is formulated by creating constraints which specify that only one out of all the sets $S_i$ of which a particular member $s$ is selected. This looks like
$$
\sum_{i,x\in S_i} s_i = 1\quad \forall x\in X
$$
where $s_i\in\{0,1\}$ indicates set $S_i$ is selected. In addition to a constraint for each member, there is an objective function which measures the cost, weight or benefit of selecting certain sets from $S$. The objective function could be any form, but we can solve linear or quadratic objective functions with Dirac. A quadratic objective function has coefficients $c_{ij},c_{i}$ for quadratic and linear terms, respectively. With the variables $s_i$, we have
$$
\sum_j\sum_i c_{ij}s_is_j + \sum_i c_is_i.
$$


## Implementation
With constraint-based solvers, it would be sufficient to implement the objective and constraints directly in the modeling or matrix format required by the solver, but an addition step is required for unconstrained solving. Penalties can be created from the constraints $As=b$ using
$$
P(s)=s^T(A^TA)s-(2b^TA)^Ts+b^Tb.
$$

When all constraints are met, $P(s)=0$ and when any constraint is violated, $P(s)>0$. There is a difficulty in combining the objective function and penalties in that a scalar value of the objective function for a constraint violating solution could be less than 0 or at least less than the penalty, which results in a value of the total function less than 0. This will result in an optimizer finding an infeasible solution, unless a multiplier is applied to $P(s)$. Sufficiently large multipliers will guarantee that no infeasible solution will take on a value of the total function which is less than any value for a feasible solution.

Here we have a value which is known to be sufficient, $\alpha$. We apply it to $P(s)$ to get the function
$$
Q(s)=\sum_j\sum_i c_{ij}s_is_j + \sum_i c_is_i + \alpha P(s)
$$

## Demonstration

In [1]:
import numpy as np
from qci_client import QciClient

### Creating the Data
$S$ is a collection of 9 different sets. The members of the sets are the letters A through F. $W$ are the weights of each subset $S_i$.

In [2]:
np.random.seed(21)
S = [{"A", "B", "C"}, {"D", "E", "F"}, {"A", "F"}, {"B", "E"}, {"C", "D"}, {"A"},
     {"B"}, {"C", "D", "E"}, {"B", "C"}]
W = [100 * np.random.random() for S_i in S]


$X$ is the union of all $S_i$.

In [3]:
X = set()
for S_i in S:
    X = X.union(S_i)
X

{'A', 'B', 'C', 'D', 'E', 'F'}

Create the constraints by indicating if a member is in a subset with a 1 in the position for the variable $s_i$ for every $x$.

In [4]:
A = []
for x in X:
    row = [1 if x in S_i else 0 for S_i in S]
    A.append(row)
A = np.array(A)
A

array([[0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 1, 1],
       [1, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0]])

In [5]:
b = np.ones((A.shape[0],))
b

array([1., 1., 1., 1., 1., 1.])

In [6]:
J = A.T@A
h = -2 * b.T@A
n = h.shape[0]
h = h.reshape((n, 1))
offset = b.T@b
J, h, offset

(array([[3, 0, 1, 1, 1, 1, 1, 1, 2],
        [0, 3, 1, 1, 1, 0, 0, 2, 0],
        [1, 1, 2, 0, 0, 1, 0, 0, 0],
        [1, 1, 0, 2, 0, 0, 1, 1, 1],
        [1, 1, 0, 0, 2, 0, 0, 2, 1],
        [1, 0, 1, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 1, 0, 1],
        [1, 2, 0, 1, 2, 0, 0, 3, 1],
        [2, 0, 0, 1, 1, 0, 1, 1, 2]]),
 array([[-6.],
        [-6.],
        [-4.],
        [-4.],
        [-4.],
        [-2.],
        [-2.],
        [-6.],
        [-4.]]),
 6.0)

Solving the problem without an objective function should reveal if there are multiple solutions to the exact cover problem.

First, create a connection to the REST API using `QciClient`.

In [7]:
client = QciClient()

The next line creates a QUBO from the quadratic and linear portions of the penalty function by adding all the linear terms in the diagonal of the quadratic operator. This file is uploaded to the API.

In [8]:
P = J + np.diag(h.T[0])
P

array([[-3.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  2.],
       [ 0., -3.,  1.,  1.,  1.,  0.,  0.,  2.,  0.],
       [ 1.,  1., -2.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  1.,  0., -2.,  0.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  0., -2.,  0.,  0.,  2.,  1.],
       [ 1.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  1.],
       [ 1.,  2.,  0.,  1.,  2.,  0.,  0., -3.,  1.],
       [ 2.,  0.,  0.,  1.,  1.,  0.,  1.,  1., -2.]])

In [9]:
file_id = client.upload_file(P, file_type="qubo")["file_id"]

Using the file ID returned by the upload method, build the job body requesting the job to run on eqc1 (aka Dirac-1), returning 5 samples.

In [10]:
body = client.build_job_body("sample-qubo", qubo_file_id=file_id, job_params={
    "sampler_type": "eqc1", "n_samples": 5
})

The job type `sample-qubo` converts the QUBO into an Ising Hamiltonian before sending to Dirac-1 for sampling.

In [11]:
response = client.process_job("sample-qubo", body)

Job submitted job_id='65a6ebcb66705e945c7914d9'-: 2024/01/16 13:49:15
RUNNING: 2024/01/16 13:49:16
COMPLETED: 2024/01/16 13:50:25


The iteration over the samples tests if the sample selected all the members of $X$ exactly once. 100% set coverage indicates that all members were included and a partition was found if no member is included in multiple sets.

In [12]:
def check_result(results, objective=None):
    samples = results["samples"]
    energies = results["energies"]
    counts = results["counts"]
    for j, sample in enumerate(samples):
        print("QUBO value:", energies[j])
        print("Times sample found", counts[j])
        sample = np.array(sample)
        if objective is not None:
            print("Objective Function value:", sample.T@objective@sample)
        print("Selected Sets")
        testX = set()
        members = []
        for i in range(len(S)):
            if sample[i] == 1:
                print(f"S_{i}", end=" ")
                testX = testX.union(S[i])
                members.extend(S[i])
        print()
        print(f"Set coverage {100*len(testX)/len(X)}%")
        print(f"Partition found: {len(testX)==len(members)}")


In [13]:
check_result(response["results"])
# save the response
penalty_only_response = response

QUBO value: -6
Times sample found 2
Selected Sets
S_2 S_6 S_7 
Set coverage 100.0%
Partition found: True
QUBO value: -6
Times sample found 2
Selected Sets
S_2 S_3 S_4 
Set coverage 100.0%
Partition found: True
QUBO value: -6
Times sample found 1
Selected Sets
S_0 S_1 
Set coverage 100.0%
Partition found: True


The objective function specified is a linear function. It is built into the diagonal of a matrix, as the linear portion of the penalty function.

In [14]:
objective = np.diag([W[i] for i in range(len(S))])
objective

array([[ 4.87248808,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , 28.91096598,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , 72.09663468,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  2.16162499,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 20.59227653,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         5.07732567,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , 30.2271894 ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0

In [15]:
alpha = np.max(objective)
alpha

72.09663468312299

In [16]:
Q = objective + alpha * P
Q

array([[-211.41741597,    0.        ,   72.09663468,   72.09663468,
          72.09663468,   72.09663468,   72.09663468,   72.09663468,
         144.19326937],
       [   0.        , -187.37893807,   72.09663468,   72.09663468,
          72.09663468,    0.        ,    0.        ,  144.19326937,
           0.        ],
       [  72.09663468,   72.09663468,  -72.09663468,    0.        ,
           0.        ,   72.09663468,    0.        ,    0.        ,
           0.        ],
       [  72.09663468,   72.09663468,    0.        , -142.03164437,
           0.        ,    0.        ,   72.09663468,   72.09663468,
          72.09663468],
       [  72.09663468,   72.09663468,    0.        ,    0.        ,
        -123.60099284,    0.        ,    0.        ,  144.19326937,
          72.09663468],
       [  72.09663468,    0.        ,   72.09663468,    0.        ,
           0.        ,  -67.01930901,    0.        ,    0.        ,
           0.        ],
       [  72.09663468,    0.        ,   

In [17]:
file_id = client.upload_file(Q, file_type="qubo")["file_id"]

In [18]:
body = client.build_job_body("sample-qubo", qubo_file_id=file_id, job_params={
    "sampler_type": "eqc1", "n_samples": 5
})

In [19]:
response = client.process_job(job_type="sample-qubo", job_body=body)

Job submitted job_id='65a6ec1366705e945c7914e6'-: 2024/01/16 13:50:27
RUNNING: 2024/01/16 13:50:28
COMPLETED: 2024/01/16 13:51:37


In [20]:
check_result(response["results"], objective)

QUBO value: -398.7963407037149
Times sample found 5
Objective Function value: 33.78345405989442
Selected Sets
S_0 S_1 
Set coverage 100.0%
Partition found: True


Check the samples from the saved response with the objective function included to manually validate the minimization.

In [21]:
check_result(penalty_only_response["results"], objective)

QUBO value: -6
Times sample found 2
Objective Function value: 168.71485354205467
Selected Sets
S_2 S_6 S_7 
Set coverage 100.0%
Partition found: True
QUBO value: -6
Times sample found 2
Objective Function value: 94.85053620146186
Selected Sets
S_2 S_3 S_4 
Set coverage 100.0%
Partition found: True
QUBO value: -6
Times sample found 1
Objective Function value: 33.78345405989442
Selected Sets
S_0 S_1 
Set coverage 100.0%
Partition found: True
